# Introduction to yabadaba: Unit conversions

This Notebook provides a quick overview of the unit conversion tools built into yabadaba.

## 1. Introduction <a id='section1'></a>

The yabadaba package provides some basic tools for handling unit conversions. The general concept of how the tools work is based on the [numericalunits](https://pypi.python.org/pypi/numericalunits) package and is as follows

1. According to the SI system of units there are seven basic units. All other units can be derived from the basic units through multiplication and division operations.
2. The majority of non-SI systems use units that can be represented as scalar factors of the SI units.
3. By knowing the multiplicative factors between all units used, input values can be converted into arbitrary working units.
4. If all inputs are in the working units, the values will be compatible with each other for any calculations performed.  This simplifies the calculations themselves as units do not need to be explicitly tracked and managed.
5. Once calculations are finished, the resulting values can be converted from the working units into whatever units you want, assuming that the output units are consistent with the output value.
6. While units are not explicitly tracked in the above steps, you can implicitly check for unit compatibility of the outputs by running the calculation again with different arbitrary working units.

The main exception to #2 above are units that are not absolute, e.g. units of temperature like Celsius and Fahrenheit.  While the units themselves are not scalar factors, units that depend on changes in temperatures will be.

While there are seven unique SI basic units (meter, kilogram, second, kelvin, ampere, candela, and mole), yabadaba uses five changeable basic units (meter, kilogram, second, kelvin, and coulomb).  Luminous intensity like candela is not yet supported, and the value of a mole is hard-set. 

**Library Imports**

In [1]:
# Standard Python libraries
import datetime

# http://www.numpy.org/
import numpy as np

# https://github.com/usnistgov/atomman
import yabadaba

from IPython.display import display, Markdown

# Show yabadaba version
print('yabadaba version =', yabadaba.__version__)

# Show date of Notebook execution
print('Notebook executed on', datetime.date.today())

yabadaba version = 0.3.0
Notebook executed on 2025-02-13


## 2. Basics <a id='section2'></a>

The yabadaba package contains unitconvert, which is an object of the UnitConverter class.  For most purposes, you should use the unitconvert object rather than initializing a new UnitConverter object.  Importing the object allows for called functions that perform unit conversions to use the same converter with the same working units settings.

In [2]:
# Get unitconvert from yabadaba
uc = yabadaba.unitconvert

### 2.1. Working units

By default, yabadaba defines working units in primary SI units.

In [3]:
print(uc.display_core_values)

m  = 1.0
kg = 1.0
s  = 1.0
C  = 1.0
K  = 1.0


### 2.2. reset_units()

If you wish, you can specify different units to work in. If you are building a yabadaba-based project, it may be useful to change the default working units for your project to be something more practical for your field. Up to four out of five of length, mass, time, energy, and charge can be directly specified to reset_units(). If less than four values are given, SI units are used for the remainders. Temperature is always 'K' when values are specified.

In [4]:
# Reset working units such that length is in 'nm', mass is in 'g' and time is in 'ps'
uc.reset_units(length='nm', mass='g', time='ps')

print(uc.display_core_values)
print()

# Show that the specified working units have conversion values of 1
print(f'nm = {uc.nm:.15}')
print(f'g = {uc.g}')
print(f'ps = {uc.ps}')

m  = 1000000000.0
kg = 1000.0
s  = 1000000000000.0
C  = 1.0
K  = 1.0

nm = 1.0
g = 1.0
ps = 1.0


If you call reset_units() with no parameters or an int seed, the working units will be set to random arbitrary values.  This can be useful when testing that your calculations are returning values in the correct unit types.

In [5]:
# Reset working units to random values
uc.reset_units()

print(uc.display_core_values)

m  = 0.0548763175722184
kg = 0.28759832478441
s  = 19.1293882002913
C  = 0.204029786326931
K  = 92.3298994737891


Working units can be reset to SI by setting seed to 'SI'

In [6]:
# Return working units to SI
uc.reset_units('SI')
print(uc.display_core_values)

m  = 1.0
kg = 1.0
s  = 1.0
C  = 1.0
K  = 1.0


### 2.3. Setting and getting static values

In design, you only need to perform unit conversions on input/output values that have units.

- For inputs, you multiply the value by the units that the value is in.

- For outputs, you divide the value by the units that you want the value in.

#### 2.3.1. Class attributes

All defined units can be accessed as attributes of unitconvert.

In [7]:
# Convert 10 angstrom^3 to working units
volume = 10 * uc.angstrom**3

# Convert to nm**3
volume_in_nm3 = volume / uc.nm**3

# Print output
print(f'10 angstrom^3 = {volume_in_nm3:.9} nm^3')

10 angstrom^3 = 0.01 nm^3


In [8]:
# Convert 5.5 kg/(m*s^2) to working units
pressure = 5.5 * uc.kg / (uc.m * uc.s**2)

# Convert to Pa
pressure_in_pa = pressure / uc.Pa

# Show that Pa = kg/(m*s^2)
print(f'5.5 kg/(m*s^2) = {pressure_in_pa:.9} Pa')

5.5 kg/(m*s^2) = 5.5 Pa


Conversions equally work on numpy arrays

In [9]:
# Convert from GPa to working units
stress = np.array([[1.1, 1.2, 1.3],
                   [1.2, 2.2, 2.3],
                   [1.3, 2.3, 3.3]]) * uc.GPa

# Print stress in MPa
print(stress / uc.MPa, 'MPa')

[[1100. 1200. 1300.]
 [1200. 2200. 2300.]
 [1300. 2300. 3300.]] MPa


#### 2.3.2. unit and parse()

For scripts where the input values and units may be user-defined, it is more convenient to be able to express the units as string values.  These string values are supported by the addition of the unit attribute and the parse() method.

- __unit__ is a dict that contains all of the defined units allowing for them to be retrieved using str names.
- __parse()__ takes a str unit term and parses it into an appropriate scalar for converting to/from the working units.  The unit term can be made up of
    - numbers
    - known unit names
    - '\*' for multiplication
    - '/' for division
    - '^' for powers
    - '(' and ')' for parenthesis
    - spaces are ignored

In [10]:
# Convert 10 angstrom^3 to working units
volume = 10 * uc.unit['angstrom']**3

# Convert to nm**3
volume_in_nm3 = volume / uc.unit['nm']**3

# Print output
print(f'10 angstrom^3 = {volume_in_nm3:.9} nm^3')

10 angstrom^3 = 0.01 nm^3


In [11]:
# Convert 5.5 kg/(m*s^2) to working units
pressure = 5.5 * uc.parse('kg/(m*s^2)')

# Convert to Pa
pressure_in_pa = pressure / uc.parse('Pa')

# Show that Pa = kg/(m*s^2)
print(f'5.5 kg/(m*s^2) = {pressure_in_pa:.9} Pa')

5.5 kg/(m*s^2) = 5.5 Pa


#### 2.3.3. set_in_units() and get_in_units()

For convenience, methods set_in_units() and get_in_units() are also defined that parse the given unit term and apply it to the given value using the correct multiplication or division.  Using these methods is recommended as it makes the code more readable and clearly shows which values are inputs and outputs.

In [12]:
# Convert 10 angstrom^3 to working units
volume = uc.set_in_units(10, 'angstrom^3')

# Convert to nm**3
volume_in_nm3 = uc.get_in_units(volume, 'nm^3')

# Print output
print(f'10 angstrom^3 = {volume_in_nm3:.9} nm^3')

10 angstrom^3 = 0.01 nm^3


In [13]:
# Convert 5.5 kg/(m*s^2) to working units
pressure = uc.set_in_units(5.5, 'kg/(m*s^2)')

# Convert to Pa
pressure_in_pa = uc.get_in_units(pressure, 'Pa')

# Show that Pa = kg/(m*s^2)
print(f'5.5 kg/(m*s^2) = {pressure_in_pa:.9} Pa')

5.5 kg/(m*s^2) = 5.5 Pa


In [14]:
# Show that conversions work with arrays
stress = uc.set_in_units(np.array([[1.1, 1.2, 1.3],
                                   [1.2, 2.2, 2.3],
                                   [1.3, 2.3, 3.3]]), 'GPa')

print(uc.get_in_units(stress, 'MPa'), 'MPa')

[[1100. 1200. 1300.]
 [1200. 2200. 2300.]
 [1300. 2300. 3300.]] MPa


#### 2.3.4. set_literal()

Another option is set_literal(), which parses a string for both values and an optional unit.  This is a convenience method for allowing user inputs in string form that may or may not contain units.  The features of set_literal() are

- Values can be parsed into single values or arrays.
- Unit is optional and separated from the value(s) by a space. All options of parse() are allowed for the unit term.

In [15]:
# Show that values without a unit are simply transformed into floats
print(uc.set_literal('102.93'))

102.93


In [16]:
# Convert 10 angstrom^3 to working units
volume = uc.set_literal('10 angstrom^3')

# Convert to nm**3
volume_in_nm3 = uc.get_in_units(volume, 'nm^3')

# Print output
print(f'10 angstrom^3 = {volume_in_nm3:.9} nm^3')

10 angstrom^3 = 0.01 nm^3


In [17]:
# Show that conversions work with arrays
stress = uc.set_literal("""[[1.1, 1.2, 1.3], 
                            [1.2, 2.2, 2.3], 
                            [1.3, 2.3, 3.3]] GPa""")

print(uc.get_in_units(stress, 'MPa'), 'MPa')

[[1100. 1200. 1300.]
 [1200. 2200. 2300.]
 [1300. 2300. 3300.]] MPa


## 3. Data model representations <a id='section3'></a>

The yabadaba package focuses on being able to easily convert between a "Python" representation and a "data model" representation in an equivalent JSON/XML format.  To this end, yabadaba recommends that values stored in the data model correspond to the following partial schema

- root element is named for the property. It can then have child elements:
    - "value" being a single float value or a 1D list of values for array elements.
    - "unit" being the str unit that the values are stored in.
    - "error" being an optional float or 1D list of floats where the error associated in the value measurement can be stored.  This is treated as having the same units as value, so it is best used for standard deviations or standard error of the mean values.
    - "shape" is an optional tuple of ints that indicates how to reshape the 1D array of values into a multi-dimensional array.

### 3.1. model()

Values can be converted into a structured data model of the above format using model().

In [18]:
# Set length as 4 nm
length = uc.set_in_units(4, 'nm')

# Transform length into a model with units in angstrom
lmodel = uc.model(length, 'angstrom')

# Print lmodel as XML
print(lmodel.xml(full_document=False))

<value>40.0</value><unit>angstrom</unit>


In [19]:
# Set list of temperatures in K
temperatures = uc.set_in_units([10,20,30,40,50], 'K')

# Transform temperatures into a model with units in K
tmodel = uc.model(temperatures, 'K')

# Print tmodel as JSON
print(tmodel.json(indent=2))

{
  "value": [
    10.0,
    20.0,
    30.0,
    40.0,
    50.0
  ],
  "unit": "K"
}


For equivalent JSON/XML representation, values with 2 or more dimensions are flattened and the shape is included in the model.

In [20]:
# Set stress tensor in 'MPa'
stress = uc.set_in_units(np.array([[1.1, 0.0, 0.0],
                                   [0.0, 2.0, 0.5],
                                   [0.0, 0.5, -1.4]]), 'MPa')
                         
# Transform stress into a model with units in kPa
smodel = uc.model(stress, 'kPa')

# Print smodel as JSON
print(smodel.json())
print()

# Print smodel as XML
print(smodel.xml(full_document=False))

{"value": [1100.0, 0.0, 0.0, 0.0, 2000.0, 500.0, 0.0, 500.0, -1400.0], "shape": [3, 3], "unit": "kPa"}

<value>1100.0</value><value>0.0</value><value>0.0</value><value>0.0</value><value>2000.0</value><value>500.0</value><value>0.0</value><value>500.0</value><value>-1400.0</value><shape>3</shape><shape>3</shape><unit>kPa</unit>


### 3.2. value_unit()

Values can then be read back in from a model, XML or JSON using value_unit().

In [21]:
# Read lmode to set length
length = uc.value_unit(lmodel)

# Print length in nm
print(uc.get_in_units(length, 'nm'), 'nm')

4.0 nm


In [22]:
# Read tmodel to set temperatures 
temperatures = uc.value_unit(tmodel)

# Print temperatures in K
print(uc.get_in_units(temperatures, 'K'), 'K')

[10. 20. 30. 40. 50.] K


In [23]:
# Read smodel to set stress 
stress = uc.value_unit(smodel)

# Print stress in 'MPa'
print(uc.get_in_units(stress, 'MPa'), 'MPa')

[[ 1.1  0.   0. ]
 [ 0.   2.   0.5]
 [ 0.   0.5 -1.4]] MPa


### 3.3. error_unit()

Standard errors associated with each given value can also be included in the model, which can then be retrieved using error_unit().

In [24]:
# Generate realistic-looking nonsense
xcoordinate = np.array([1, 2, 3, 4, 5]) + 0.2 * np.random.rand(5) - 0.1
xcoorderror = np.array([0.2, 0.2, 0.2, 0.2, 0.2]) + 0.02 * np.random.rand(5) - 0.01

# Assign units to nonsense
xcoordinate = uc.set_in_units(xcoordinate, 'cm')
xcoorderror = uc.set_in_units(xcoorderror, 'cm')

# Generate model of nonsense with error
model = uc.model(xcoordinate, 'm', error=xcoorderror)
print(model.json(indent=2))

{
  "value": [
    0.009054388514915858,
    0.01907442470978317,
    0.03088364889121383,
    0.04001173839756155,
    0.049144173026118454
  ],
  "error": [
    0.001973042508968387,
    0.0019133955434186552,
    0.0019413789152596644,
    0.0019132831280297207,
    0.001979651674322935
  ],
  "unit": "m"
}


Errors can then be similarly extracted from the model using error_unit()

In [25]:
# Read realistic-looking nonsense back in
print('value =', uc.get_in_units(uc.value_unit(model), 'mm'), 'mm')
print('error =', uc.get_in_units(uc.error_unit(model), 'mm'), 'mm')

value = [ 9.05438851 19.07442471 30.88364889 40.0117384  49.14417303] mm
error = [1.97304251 1.91339554 1.94137892 1.91328313 1.97965167] mm


## 4. Unit debugging <a id='section4'></a>

There is no explicit unit control with unitconvert, but correct unit conversions can still be debugged and tested by seeing if changing the working units changes values.  You can do this testing in one of two ways:

- Run the calculation, call reset_units(), then run again
- Initialize a new UnitConverter with different basic unit settings.

In [26]:
# Create an alternate unitconverter with random units
uc_alt = yabadaba.UnitConverter.UnitConverter()
uc_alt.reset_units()

In [27]:
# Perform valid conversion of atmosphere into GPa
pressure = uc.set_in_units(57, 'atm')
pressure_in_GPa = uc.get_in_units(pressure, 'GPa')
print(f'57 atm = {pressure_in_GPa} GPa')

# Perform the same valid conversion with uc_alt = same results
pressure = uc_alt.set_in_units(57, 'atm')
pressure_in_GPa = uc_alt.get_in_units(pressure, 'GPa')
print(f'57 atm = {pressure_in_GPa} GPa')

57 atm = 0.005775525 GPa
57 atm = 0.005775525 GPa


In [28]:
# Perform invalid conversion of seconds into GPa
time = uc.set_in_units(57, 's')
time_in_GPa = uc.get_in_units(time, 'GPa')
print(f'57 s = {time_in_GPa} GPa')

# Perform the same invalid conversion with uc_alt = different results!
time = uc_alt.set_in_units(57, 's')
time_in_GPa = uc_alt.get_in_units(time, 'GPa')
print(f'57 s = {time_in_GPa} GPa')

57 s = 5.7e-08 GPa
57 s = 8.952551978732456e-11 GPa


## 5. Recognized units

The unitdoc attribute returns a dict of all defined unit terms and their names/descriptions. This can be used to quickly check if a unit is what you think it is, or to generate a list of all recognized units.

In [29]:
markdown_str = '### List of recognized units\n'

for unit in uc.unitdoc:
    markdown_str += f'- __{unit}__: {uc.unitdoc[unit]}\n'

display(Markdown(markdown_str))    

### List of recognized units
- __A__: ampere
- __ARichardson__: Richardson constant
- __Ah__: amp-hour
- __C__: coulomb
- __Da__: dalton
- __F__: farad
- __G__: gauss
- __GHz__: gigahertz
- __GHz·2π__: gigahertz times 2 pi
- __GJ__: gigajoule
- __GL__: gigaliter
- __GN__: giganewton
- __GNewton__: Gravitational constant
- __GPa__: gigapascal
- __GV__: gigavolt
- __GW__: gigawatt
- __Gal__: galileo
- __GeV__: gigaelectron volt
- __Gohm__: gigaohm
- __Gpc__: gigaparsec
- __GΩ__: gigaohm
- __H__: henry
- __Hartree__: Hartree energy, approximately 27.2 eV
- __Hz__: hertz
- __Hz·2π__: hertz times 2 pi
- __J__: joule
- __K__: kelvin
- __KJos__: Josephson constant
- __L__: liter
- __M__: molar
- __MEarth__: mass of earth
- __MHz__: megahertz
- __MHz·2π__: megahertz times 2 pi
- __MJ__: megajoule
- __ML__: megaliter
- __MN__: meganewton
- __MPa__: megapascal
- __MV__: megavolt
- __MW__: megawatt
- __Mbar__: megabar
- __MeV__: megaelectron volt
- __Mohm__: megaohm
- __Mpc__: megaparsec
- __Msolar__: mass of the sun)
- __MΩ__: megaohm
- __N__: newton
- __NA__: Avogadro's number
- __Oe__: oersted
- __PHz__: petahertz
- __PHz·2π__: petahertz times 2 pi
- __Pa__: pascal
- __Phi0__: magnetic flux quantum
- __REarth__: radius of earth
- __RKlitz__: von Klitzing constant
- __Rgas__: ideal gas constant (see README)
- __Rinf__: Rydberg constant
- __Ry__: Rydberg energy, approximately 13.6 eV
- __S__: siemens
- __T__: tesla
- __THz__: terahertz
- __THz·2π__: terahertz times 2 pi
- __TV__: teravolt
- __TW__: terawatt
- __TeV__: teraelectron volt
- __V__: volt
- __W__: watt
- __Wb__: weber
- __Wh__: watt-hour
- __Z0__: vacuum impedance, 377 ohms
- __aBohr__: Bohr radius
- __aF__: attofarad
- __aL__: attoliter
- __alphaFS__: fine-structure constant
- __amu__: atomic mass unit
- __angstrom__: angstrom
- __astro_unit__: astronomical unit
- __atm__: standard atmosphere pressure
- __bar__: bar
- __btu__: British thermal unit
- __c0__: speed of light in vacuum
- __cbar__: centibar
- __cm__: centimeter
- __day__: solar day
- __dbar__: decibar
- __debye__: debye dipole moment, approximately 0.0208 e nm
- __degCinterval__: temperature difference in degrees Celsius
- __degFinterval__: temperature difference in degrees Fahrenheit
- __dyn__: dyne
- __e__: charge of proton
- __eV__: electron volt
- __eotvos__: eotvos
- __eps0__: electric constant, permittivity of vacuum
- __erg__: erg
- __fA__: femtoampere
- __fF__: femtofarad
- __fJ__: femtogram
- __fL__: femtoliter
- __fM__: femtomolar
- __fN__: femtonewton
- __fg__: femtogram
- __fm__: femtometer
- __fmol__: femtomole
- __foot__: foot
- __fs__: femtosecond
- __g__: gram
- __g0__: standard earth gravitational acceleration
- __hPa__: hectopascal
- __hPlanck__: planck constant
- __hbar__: reduced planck constant
- __horsepower_imperial__: imperial horsepower
- __horsepower_metric__: metric horsepower
- __hour__: hour
- __inch__: inch
- __kB__: Boltzmann constant
- __kDa__: kilodalton
- __kG__: kilogauss
- __kHz__: kilohertz
- __kHz·2π__: kilohertz times 2 pi
- __kJ__: kilojoule
- __kL__: kiloliter
- __kN__: kilonewton
- __kPa__: kilopascal
- __kV__: kilovolt
- __kW__: kilowatt
- __kWh__: kilowatt-hour
- __kbar__: kilobar
- __kcal__: kilocalorie (a.k.a large Calorie, dietary Calorie)
- __keV__: kiloelectron volt
- __kg__: kilogram
- __km__: kilometer
- __kohm__: kiloohm
- __kpc__: kiloparsec
- __kΩ__: kiloohm
- __lbf__: pound-force (international avoirdupois pound)
- __lbm__: pound mass (international avoirdupois pound)
- __lightyear__: lightyear
- __m__: meter
- __mA__: milliampere
- __mAh__: milliamp-hour
- __mC__: millicoulomb
- __mG__: milligauss
- __mGal__: milligalileo
- __mH__: millihenry
- __mHz__: megahertz
- __mHz·2π__: millihertz times 2 pi
- __mJ__: milligram
- __mK__: millikelvin
- __mL__: milliliter
- __mM__: millimolar
- __mN__: millinewton
- __mS__: millisiemens
- __mT__: millitesla
- __mV__: millivolt
- __mW__: milliwatt
- __mbar__: millibar
- __me__: electron mass
- __meV__: millielectron volt
- __mg__: milligram
- __mile__: mile
- __minute__: minute
- __mm__: millimeter
- __mmol__: millimole
- __mn__: neutron mass
- __mohm__: milliohm
- __mol__: mole
- __mp__: proton mass
- __ms__: millisecond
- __mtorr__: millitorr
- __mu0__: magnetic constant, permeability of vacuum
- __mΩ__: milliohm
- __nA__: nanoampere
- __nC__: nanocoulomb
- __nF__: nanofarad
- __nH__: nanohenry
- __nJ__: nanogram
- __nK__: nanokelvin
- __nL__: nanoliter
- __nM__: nanomolar
- __nN__: nanonewton
- __nS__: nanosiemens
- __nT__: nanotesla
- __nV__: nanovolt
- __nW__: nanowatt
- __ng__: nanogram
- __nm__: nanometer
- __nmol__: nanomole
- __ns__: nanosecond
- __ohm__: ohm
- __pA__: picoampere
- __pF__: picofarad
- __pJ__: picogram
- __pK__: picokelvin
- __pL__: picoliter
- __pM__: picomolar
- __pN__: piconewton
- __pW__: picowatt
- __pc__: parsec
- __pg__: picogram
- __pi__: pi
- __pm__: picometer
- __pmol__: picomole
- __ps__: picosecond
- __psi__: pounds force per square inch
- __rpm__: revolutions per minute
- __rpm·2π__: revolutions per minute times 2 pi
- __rtHz__: root Hertz
- __s__: second
- __sigmaSB__: Stefan-Boltzmann constant
- __smallcal__: small calorie (a.k.a. gram calorie)
- __thou__: thousandth of an inch, mil
- __tonne__: tonne
- __torr__: torr
- __uA__: microampere
- __uBohr__: Bohr magneton
- __uC__: microcoulomb
- __uF__: microfarad
- __uG__: microgauss
- __uGal__: microgalileo
- __uH__: microhenry
- __uJ__: microgram
- __uK__: microkelvin
- __uL__: microliter
- __uM__: micromolar
- __uN__: micronewton
- __uNuc__: nuclear magneton
- __uS__: microsiemens
- __uT__: microtesla
- __uV__: microvolt
- __uW__: microwatt
- __ug__: microgram
- __um__: micrometer
- __umol__: micromole
- __us__: microsecond
- __week__: week
- __year__: sidereal year
- __Å__: angstrom
- __ħ__: reduced planck constant
- __Ω__: ohm
- __αFS__: fine-structure constant
- __ε0__: electric constant, permittivity of vacuum
- __μ0__: magnetic constant, permeability of vacuum
- __π__: pi
- __σSB__: Stefan-Boltzmann constant
